In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.ticker as ticker
from IPython.display import Latex, display, Math
import numpy as np
import pandas as pd
from tqdm import tqdm
import math
from torch.utils.data import DataLoader
from functools import partial
from argparse import ArgumentParser

In [6]:
data_path = "../OneDrive_1_2022-12-13/Table "
data_prefix1 =  "1 - Done/"
data_prefix2 =  "2 - Done/"
data_prefix3 =  "3 - Done/"
dataset_perfix = '../Datasets/'
fig_save_path = './figures/'
gif_save_path = fig_save_path + 'gif/'
total_save_path = fig_save_path + 'total/'
index_name_list_2 = ["ChEMBL_Target_ID","UniProt_ID","ChEMBL_Compound_ID","ChEMBL_Compound_Key","Standard_Type","Standard_Relation","Standard_Value","Standard_Units","Standard_Flag","ChEMBL_Assay_ID"]
index_name_list_3 = ['ChEMBL_Compound_ID', '2D_graph', 'SMILES', 'Standard_InChI', 'Standard_InChI_Key']#, 'MolFile'

# Download protein sequences according to chembl id

In [5]:
from chembl_webresource_client.new_client import new_client
target = new_client.target
target_query = target.search('CHEMBL1824')[0]
target_components = target.target_components.filter(target__chembl_id=target_query['target_chembl_id']).all()
for component in target_components:
    print(component.accession)

AttributeError: 'QuerySet' object has no attribute 'target_components'

In [8]:
help(target_query.keys)

Help on built-in function keys:

keys(...) method of builtins.dict instance
    D.keys() -> a set-like object providing a view on D's keys



In [73]:
for file_name in tqdm(os.listdir(data_path + data_prefix2)):
    target_id = file_name.split("-")[0][:-1]
    table2_df = pd.read_csv(data_path + data_prefix2 + target_id + ' - table 2.csv', header=0, names = index_name_list_2,usecols=range(len(index_name_list_2)))
    table3_df = pd.read_csv(data_path + data_prefix3 + target_id + ' - table 3.csv', header=0, names = index_name_list_3,usecols=range(len(index_name_list_3)))
    table2_df = table2_df.drop(np.where(np.isnan(table2_df['Standard_Value']))[0])
    table2_df = table2_df.reset_index(drop=True)
    table2_group = table2_df.groupby('Standard_Type')
    for key, value in table2_group:
        key = key.replace('/', '_divide_')
        key = key.replace('=<', '_leq_')
        # print(key)
        value = value.reset_index(drop=True)
        value['Standard_Type'] = key
        value_group = value.groupby('Standard_Units')
        for unit_key, unit_value in value_group:
            if len(unit_value) < 6:
                continue
            unit_key = unit_key.replace('/', '_divide_')
            unit_key = unit_key.replace('*', '_x_')
            unit_value['Standard_Units'] = unit_key
            unit_value = unit_value.reset_index(drop=True)
            if key == 'Ratio' or unit_value['Standard_Units'][0] == '%':
                unit_value['Standard_Value'] = unit_value['Standard_Value']/100
                unit_value['Standard_Units'] = 'times'
            if unit_value['Standard_Units'][0] == 'nM':
                unit_value['Standard_Value'] = -np.log(unit_value['Standard_Value']+1e-6)
                unit_value['Standard_Units'] = '-log(nM)'
            dataset_save_path = dataset_perfix + target_id + '/'+ key + '_in_' + unit_key + '/'
            if not os.path.exists(dataset_save_path):
                os.makedirs(dataset_save_path)
            Compound2_list, index2_list, index3_list, drop_index2 = [[], [], [], []]
            counter2_dict = {}
            for index2, ChEMBL_Compound_ID in enumerate(unit_value['ChEMBL_Compound_ID']):
                if ChEMBL_Compound_ID not in Compound2_list:
                    index3 = np.argwhere(table3_df['ChEMBL_Compound_ID'].values==ChEMBL_Compound_ID)[0]
                    counter2_dict[ChEMBL_Compound_ID] = [index2]
                    Compound2_list.append(ChEMBL_Compound_ID)
                    index2_list.append(index2)
                    index3_list.extend(index3)
                else:
                    first_index2 = counter2_dict[ChEMBL_Compound_ID][0]
                    index2_len = len(counter2_dict[ChEMBL_Compound_ID])
                    unit_value['Standard_Value'].values[first_index2] = (index2_len * unit_value['Standard_Value'].values[first_index2] + unit_value['Standard_Value'].values[index2])/index2_len
                    counter2_dict[ChEMBL_Compound_ID] += [index2]
                    drop_index2.append(index2)
            # print(drop_index2)
            unit_value = unit_value.drop(drop_index2)
            unit_value = unit_value.reset_index(drop=True)
            if len(unit_value) < 6:
                continue
            # print(target_id, key, unit_key, index3_list, unit_value, table3_df.iloc[index3_list, 2].values)
            for i, index_name_3 in enumerate(index_name_list_3):
                unit_value[index_name_3] = table3_df.iloc[index3_list, i].values
            unit_value.to_csv(path_or_buf=dataset_save_path+'/'+target_id+'_'+key+'_in_'+unit_key+'_all.csv', index=False)

  0%|▎                                                                               | 4/901 [04:13<8:35:17, 34.47s/it]C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_24252\663596540.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  table2_df = pd.read_csv(data_path + data_prefix2 + target_id + ' - table 2.csv', header=0, names = index_name_list_2,usecols=range(len(index_name_list_2)))
  4%|███▍                                                                           | 39/901 [46:14<2:07:27,  8.87s/it]C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_24252\663596540.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  table2_df = pd.read_csv(data_path + data_prefix2 + target_id + ' - table 2.csv', header=0, names = index_name_list_2,usecols=range(len(index_name_list_2)))
  7%|█████▎                                                                       

In [ ]:
!python ./Data_Analysis/splitter.py

 55%|██████████████████████                  | 495/899 [00:01<00:00, 426.48it/s]Delete empty fold: ./Datasets/CHEMBL5393/Intrinsic_activity_in_pmol
Delete empty fold: ./Datasets/CHEMBL5393/.ipynb_checkpoints
Processing dgl graphs from scratch...
Start initializing RDKit molecule instances...
Start computing Bemis-Murcko scaffolds.
Processing dgl graphs from scratch...
Start initializing RDKit molecule instances...
Start computing Bemis-Murcko scaffolds.
Delete empty fold: ./Datasets/CHEMBL5393/permeability_in_uL_divide_hr_divide_cm2
Processing dgl graphs from scratch...
Start initializing RDKit molecule instances...
Start computing Bemis-Murcko scaffolds.
Processing dgl graphs from scratch...
Start initializing RDKit molecule instances...
Start computing Bemis-Murcko scaffolds.
Processing dgl graphs from scratch...
Start initializing RDKit molecule instances...
Start computing Bemis-Murcko scaffolds.
Processing dgl graphs from scratch...
Start initializing RDKit molecule instances...
S

In [100]:
all_num = 0
test_num = 0
for target_id in tqdm(os.listdir(dataset_perfix)):
    for assay_type in os.listdir(dataset_perfix + target_id):
        dataset_path = dataset_perfix + target_id + '/' + assay_type + '/' + target_id + '_' + assay_type
        all_df = pd.read_csv(dataset_path + '_all.csv', header=0, names = ['value'],usecols=[6])
        test_df = pd.read_csv(dataset_path + '_test.csv', header=0, names = ['value'],usecols=[0])
        all_num += len(all_df)
        test_num += len(test_df)
print(all_num, test_num)

 54%|██████████████████████████████████████████▉                                     | 483/899 [00:49<00:42,  9.76it/s]


FileNotFoundError: [Errno 2] No such file or directory: '../Datasets/CHEMBL3371/Activity_in_a.u/CHEMBL3371_Activity_in_a.u_all.csv'

In [3]:
for target_id in tqdm(os.listdir('../Results')):
    for assay_type in os.listdir('../Results/' + target_id):
        for model in os.listdir('../Results/' + target_id + '/' + assay_type):
            !rm -r '../Results/' + target_id + '/' + assay_type + '/' + model + '/graph.bin'

  0%|                                                                                             | 0/7 [00:00<?, ?it/s]

rm: cannot remove '+': No such file or directory
rm: cannot remove 'target_id': No such file or directory
rm: cannot remove '+': No such file or directory
rm: it is dangerous to operate recursively on '/'
rm: use --no-preserve-root to override this failsafe
rm: cannot remove '+': No such file or directory
rm: cannot remove 'assay_type': No such file or directory
rm: cannot remove '+': No such file or directory
rm: it is dangerous to operate recursively on '/'
rm: use --no-preserve-root to override this failsafe
rm: cannot remove '+': No such file or directory
rm: cannot remove 'model': No such file or directory
rm: cannot remove '+': No such file or directory
rm: cannot remove '/graph.bin': No such file or directory
rm: cannot remove '../Results/': No such file or directory
rm: cannot remove '+': No such file or directory
rm: cannot remove 'target_id': No such file or directory
rm: cannot remove '+': No such file or directory
rm: it is dangerous to operate recursively on '/'
rm: use --

  0%|                                                                                             | 0/7 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '../Results/CHEMBL5414/Ki_in_nM'

In [90]:
import requests
from tqdm import tqdm
import os
import urllib.request
from chembl_webresource_client.new_client import new_client
import urllib.parse
import urllib.request
import urllib
from bs4 import BeautifulSoup
import requests

headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0'}
save_dir = '/home/yueming/Drug_Discovery/Baselines/PointSite-master/data/pdb/'
saved_list = os.listdir(save_dir)
saved_chembl_list = [name.split("_")[0] for name in saved_list]
for file_name in tqdm(os.listdir(data_path + data_prefix2)):
    chembl_id = file_name.split("-")[0][:-1]
    if chembl_id in saved_chembl_list:
        continue
    target = new_client.target
    protein = target.filter(chembl_id=chembl_id)
    UniProtID = protein[0]['target_components'][0]['accession']
    print(chembl_id, UniProtID)
    continue
    req = requests.get(f"https://rest.uniprot.org/uniprotkb/search?query={UniProtID}&fields=structure_3d").json()
    try:
        pdb_id = req['results'][0]['uniProtKBCrossReferences'][0]['id']
        f = urllib.request.urlopen("http://www.rcsb.org/pdb/files/"+pdb_id+".pdb")
        fname = save_dir + f'{chembl_id}_{UniProtID}_{pdb_id}.pdb'
        with open(fname,"wb") as g:
            g.write(f.read())  
        print(chembl_id, UniProtID, pdb_id)
    except:
        for v in range(10):
            version = 10 - v
            AF_name = f'AF-{UniProtID}-F1-model_v{version}'
            url = f'https://alphafold.ebi.ac.uk/files/{AF_name}.pdb'
            r = requests.get(url, headers=headers, verify=False)
            fname_af = save_dir + f'{chembl_id}_{UniProtID}_{AF_name}.pdb'
            r = r.text.splitlines() 
            if len(r) > 10:
                with open(fname_af,'w') as files:
                    for lines in r:
                        files.write(lines)
                        files.write('\n')
                if version == 10:
                    print(chembl_id, UniProtID, AF_name)
                break

100%|██████████████████████████████████████████████████████████████████████████████| 900/900 [00:00<00:00, 34644.58it/s]

CHEMBL1293269 A3EZJ3
CHEMBL3471 Q7ZJM1
CHEMBL2146310 Q77YF9
CHEMBL5375 Q8JXU8
CHEMBL6135 B4URF0
CHEMBL2095231 D2K2A8
CHEMBL1293303 Q194T2
CHEMBL2857 Q4U254
CHEMBL2366517 Q9YQ12
CHEMBL3307224 Q5L478


In [93]:
save_dir = '/home/yueming/Drug_Discovery/Baselines/PointSite-master/data/pdb/'
saved_list = os.listdir(save_dir)

# 保存列表到文件
with open('/home/yueming/Drug_Discovery/Baselines/PointSite-master/data/blind_list.txt', 'w') as f:
    for item in saved_list:
        f.write(str(item[:-4]) + '\n')


In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem

save_path = '/home/yueming/Drug_Discovery/Baselines/PointSite-master/data/sdf/'
dock_df = pd.read_csv('dock.csv')
mols, smiles, sid, libs = [], [], [], []
for col in dock_df.columns:
    for i, smi in enumerate(dock_df[col].values):
        if isinstance(smi, str):
            smiles.append(smi)
            libs.append(col)
            sid.append(i)
            mols.append(Chem.MolFromSmiles(smi))
            
hmols = [Chem.AddHs(m) for m in mols]
for n, mol in enumerate(hmols):
    try:
        AllChem.EmbedMolecule(mol,AllChem.ETKDG())
        print(AllChem.UFFOptimizeMolecule(mol,1000))
        hmols[n].SetProp("_Task","%s"%libs[n])
        hmols[n].SetProp("_CompoundID","%s"%sid[n])
        hmols[n].SetProp("_SMILES","%s"%smiles[n])
        writer = Chem.SDWriter(save_path + f'{libs[n]}_{sid[n]}.sdf')
        writer.write(hmols[n])
        writer.close()
    except:
        print(f'Failed to process {libs[n]} {sid[n]}')
        

Failed to process CHEMBL1821_Ki_in_nM 0
Failed to process CHEMBL1821_Ki_in_nM 1
Failed to process CHEMBL1821_Ki_in_nM 2
Failed to process CHEMBL1821_Ki_in_nM 3
Failed to process CHEMBL1821_Ki_in_nM 4
Failed to process CHEMBL1821_Ki_in_nM 5
Failed to process CHEMBL1821_Ki_in_nM 6
Failed to process CHEMBL1821_Ki_in_nM 7
Failed to process CHEMBL1821_Ki_in_nM 8
Failed to process CHEMBL1821_Ki_in_nM 9
Failed to process CHEMBL1821_Ki_in_nM 10
Failed to process CHEMBL1821_Ki_in_nM 11
Failed to process CHEMBL1821_Ki_in_nM 12
Failed to process CHEMBL1821_Ki_in_nM 13
Failed to process CHEMBL1821_Ki_in_nM 14
Failed to process CHEMBL1821_Ki_in_nM 15
Failed to process CHEMBL1821_Ki_in_nM 16
Failed to process CHEMBL1821_Ki_in_nM 17
Failed to process CHEMBL1821_Ki_in_nM 18
Failed to process CHEMBL1821_Ki_in_nM 19
Failed to process CHEMBL1821_Ki_in_nM 20
Failed to process CHEMBL1821_Ki_in_nM 21
Failed to process CHEMBL1821_Ki_in_nM 22
Failed to process CHEMBL1821_Ki_in_nM 23
Failed to process CHEMBL18

[16:00:48] UFFTYPER: Unrecognized charge state for atom: 54
[16:00:48] UFFTYPER: Unrecognized charge state for atom: 54


Failed to process CHEMBL202_Ki_in_nM 10
0
Failed to process CHEMBL202_Ki_in_nM 12


[16:00:51] UFFTYPER: Warning: hybridization set to SP3 for atom 16
[16:00:51] UFFTYPER: Warning: hybridization set to SP3 for atom 27
[16:00:51] UFFTYPER: Warning: hybridization set to SP3 for atom 36
[16:00:51] UFFTYPER: Warning: hybridization set to SP3 for atom 45
[16:00:51] UFFTYPER: Warning: hybridization set to SP3 for atom 54
[16:00:51] UFFTYPER: Warning: hybridization set to SP3 for atom 63
[16:00:51] UFFTYPER: Warning: hybridization set to SP3 for atom 72
[16:00:51] UFFTYPER: Warning: hybridization set to SP3 for atom 76
[16:00:51] UFFTYPER: Warning: hybridization set to SP3 for atom 1
[16:00:51] UFFTYPER: Warning: hybridization set to SP3 for atom 17
[16:00:51] UFFTYPER: Warning: hybridization set to SP3 for atom 28
[16:00:51] UFFTYPER: Warning: hybridization set to SP3 for atom 37
[16:00:51] UFFTYPER: Warning: hybridization set to SP3 for atom 46
[16:00:51] UFFTYPER: Warning: hybridization set to SP3 for atom 55
[16:00:51] UFFTYPER: Warning: hybridization set to SP3 for atom

Failed to process CHEMBL202_Ki_in_nM 13
0
0
0
1
0
0
